In [1]:
#CHANGE THE PATH TO THE PATH AT YOUR PC!
path = "C:\\Users\\user\\Desktop\\TUe\\Topological\\Project\\Geolife Trajectories 1.3\\Data\\"
CITY_BORDERS = [40.19, 39.65, 115.98, 116.74]
crop_to_borders = True
THRESHOLD_VERTICES = 50
THRESHOLD_EDGES = 4
GRID_SHAPE = (40, 40)

import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from Graph import *
from Grid import *

In [19]:
df = pd.read_csv(".\\presets\\very_short.csv")
df = df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)

In [20]:
df

,latitude,longitude,altitude,date,trajectory,user,transportation,change
0,39.994622,116.326757,150.0,2008-10-27 19:54:52,0.200810,0,NaN,NaN
1,39.994614,116.326751,150.0,2008-10-27 19:54:54,0.200810,0,NaN,NaN
2,39.994602,116.326769,150.0,2008-10-27 19:54:59,0.200810,0,NaN,NaN
3,39.994599,116.326769,150.0,2008-10-27 19:55:04,0.200810,0,NaN,NaN
4,39.994599,116.326769,150.0,2008-10-27 19:55:07,0.200810,0,NaN,NaN
...,...,...,...,...,...,...,...,...
103797,39.986867,116.304267,65.0,2008-03-16 10:12:08,99.200803,99,NaN,0.000700
103798,39.987133,116.305183,42.0,2008-03-16 10:13:19,99.200803,99,NaN,0.000955
103799,39.987633,116.305267,90.0,2008-03-16 10:14:26,99.200803,99,NaN,0.000507
103800,39.988183,116.306317,37.0,2008-03-16 10:16:10,99.200803,99,NaN,0.001185


In [21]:
#Grid creation example:
grid = Grid(30, 30) 
print(grid)
print(grid.cell_list[0].number_of_locations) #should be 0

Grid of side length (latitude) 0.018 and (longitude) 0.02533 and total number of cells: 900
0


In [22]:
def grid_graphify(df: pd.DataFrame, crop_to_borders: bool, grid_shape: tuple, 
                  THRESHOLD_VERTICES: int, THRESHOLD_EDGES: int, CITY_BORDERS: list):
    '''
    For details, check Section 4.2 of the report
    '''
    
    #(possibly) cropping the data to the city borders:
    if crop_to_borders:
        mask_north = df['latitude'] <= CITY_BORDERS[0]
        mask_south = df['latitude'] >= CITY_BORDERS[1]
        mask_east = df['longitude'] <= CITY_BORDERS[3]
        mask_west = df['longitude'] >= CITY_BORDERS[2]
        mask_all = mask_north & mask_south & mask_east & mask_west
        df = df[mask_all].reset_index(drop=True)
        
    #Grid creation example:
    grid = Grid(grid_shape[0], grid_shape[1]) 
    print(grid)
    print(grid.cell_list[0].number_of_locations) #should be 0
    
    #Handling the cell number per row:
    df['cell_number'] = [0] * df.shape[0] #initializing to 0 in every row
    df['cell_number'] = df.apply(lambda x: 
                             latitude_to_cell_nr(x['latitude'], grid.longitude_cells_nr, 
                                                 grid.latitude_cell_length, CITY_BORDERS)\
                             + longitude_to_cell_nr(x['longitude'], grid.latitude_cell_length, CITY_BORDERS), 
                             axis=1)
    
    print(df.shape)
    
    return df
    
    #This actually performs the large computation:
    grid.feed_list_of_cell_numbers(list(df.cell_number))

    #And this just switches some flags around:
    grid.vertexize(THRESHOLD_VERTICES)
    
    #By grouping per trajectory, we can find the unique vertices each trajectory has traversed!
    df_grouped = df.groupby('trajectory')
    print(grid.cell_list[12].is_vertex)
    
    #{(213, 7): 5, ...} meaning there are 5 trajectories that went from cell 213 to cell 7 immediately
    edges_dict = find_subsequent_vertex_pairs(df_grouped, grid)
    
    #Same as above, but filtered by a threshold
    edges = [key for key in edges_dict if edges_dict[key] > THRESHOLD_EDGES]
    print(edges[:5])
    
    g = rav_graph_to_sven_graph_2(grid, edges)
    
    g.remove_unconnected_vertices()
    
    return g

In [23]:
df = grid_graphify(df, True, GRID_SHAPE, THRESHOLD_EDGES, THRESHOLD_VERTICES, CITY_BORDERS)
#g.save('random_graph_4')

Grid of side length (latitude) 0.0135 and (longitude) 0.019 and total number of cells: 1600
0
(90404, 9)


In [25]:
df['cell_number'].unique()

array([1025, 1065,  985,  944,  984,  749,  750,  737,  736,  664,  826,
        827, 1105, 1026, 1066, 1064,  986,  987,  745,  746,  706,  666,
        626,  586,  545,  505,  465,  424,  344,  303,  263,  264,  224,
        225,  185,  186,  147,  108,   69,   70,   32,  867,  866,  788,
        988,  908,  868,  833,  870,  910,  752, 1029, 1069,  824, 1101,
        946, 1106,  990, 1107,  951,  789,  904,  914,  945,  630,  629,
        907,  983,  943,  864,  823,  783,  794,  903, 1023, 1024,  905,
        828,  829,  830,  865,  825,  863,  834,  835,  795,  947,  844,
        948,  949,  909,  869,  912,  872,  950,  952,  911, 1324, 1284,
       1323,  791,  906,  902,  901,  861,  860,  859,  819,  818, 1071,
       1070,  982,  942,  941,  940,  938,  978, 1017, 1016, 1015,  975,
        974, 1114, 1113, 1110, 1109,  989,  805,  806,  766,  767,  881,
        920, 1077, 1076, 1075, 1074, 1224, 1264, 1265,  915,  875,  799,
        800,  755,  801,  761,  804,  803,  796,  7

Determining whether the cell is or is not a vetex - for all cells in the grid:

In [ ]:
# def plot_cell_containment(grid: Grid, threshold: int, bins = 20):
#     ''''''
#     wow = pd.Series([grid.cell_list[i].number_of_locations for i in range(grid.latitude_cells_nr*grid.longitude_cells_nr)])
#     wow = pd.Series([wow[i] for i in range(len(wow)) if wow[i] > threshold])
#     ax = wow.plot(kind='hist', bins=bins)
#     ax.set_xlabel("# GPS locations per cell of the grid")
#     ax.set_ylabel("# cells with such # locations")
    
# plot_cell_containment(grid, threshold=THRESHOLD_VERTICES)

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))
ax.scatter(df.latitude, df.longitude, zorder=1, alpha= 0.2, c='b', s=10)

# for edge in g.edges:
#     lat1 = min(edge.vertices).latitude
#     long1 = min(edge.vertices).longitude
    
#     lat2 = max(edge.vertices).latitude
#     long2 = max(edge.vertices).longitude
    
#     plt.plot([lat1, lat2], [long1, long2], marker = 'o')

# star_vertices = [vertex for vertex in graph.vertices if len(vertex.edges) == 1]
# for vertex in star_vertices:
#     ax.plot(vertex.latitude, vertex.longitude, markersize=50)
    
ax = plt.gca()
#ax.set_xlim([40.0035, 40.0115])
#ax.set_ylim([116.3175, 116.3245])
print(f"reduced from {len(df)} to {len(g.vertices)} points")